## Import Packages and Routines

In [ ]:
import numpy as np
import scipy as sc
import math
import matplotlib
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse import diags

## Sorceror Duel: Markov Process Example

Input the transition matrix and initial conditions 

In [ ]:
P = np.array([[0,0.5,0,0], [2./3.,0,0,0],[1./3.,0,1,0],[0,0.5,0,1]])
print P
x0 = np.array([1.,0,0,0])
print x0

Compute the probability vector after alpha turns

In [ ]:
alpha = 5
Palpha = np.linalg.matrix_power(P,alpha)
xalpha = np.dot(Palpha,x0)
print xalpha

## 1D Problem: -u''+u = f(x), u(x) 2 pi periodic

f(x) chosen to match a given exact solution u(x) for testing purposes

Second order finite difference method is used 

The error should behave like O(1/N^2) where N is the number of grid points 

### Input and solve a 1D boundary value problem as a full matrix

Solve the system, plot the results, compare to the exact solution 

In [ ]:
N = 16;
pi = math.pi
h = 2*pi/N     # grid spacing
fact = 1/(h**2)
ond = np.zeros(N)+2*fact+1;
offd = np.zeros(N-1)-fact;
A = np.diag(ond,0)+np.diag(offd,1)+np.diag(offd,-1) # matrix is mostly tridiagonal
A[0,N-1] = -fact # terms from periodicity
A[N-1,0] = -fact
x = np.arange(N)*h
uexact = np.zeros(N) # exact solution
rhs = np.zeros(N) # RHS f that matches the exact solution
for j in range(0,N): # could not figure out how to do this as a vector operation
    uexact[j] = math.exp(math.cos(x[j]))
    rhs[j] = uexact[j]*(math.cos(x[j])+math.cos(x[j])**2) 
u = np.linalg.solve(A, rhs) # solve 
plt.figure(1)
plt.plot(x,uexact,x,u) # plot the exact and computed solutions 
# I could not figure out how to attach a legend
plt.show()
error = np.amax(np.absolute(u-uexact)) # maximum norm of the error
print 'Maximum norm error %10.4e' % error

## Input and solve the same 1D boundary value problem as a sparse matrix

In [ ]:
N = 1000;
pi = math.pi
h = 2*pi/N
fact = 1/(h**2)
A = csr_matrix((N,N))
x = np.arange(N)*h
uexact = np.zeros(N)
rhs = np.zeros(N)
# This loop is inefficient but I could not find a way to input the sparse matrix as diagonals
for j in range(0,N):  # loop by matrix row
    A[j,j] = 2*fact+1 # diagonals
    if j==0:
        jm = N-1
    else:
        jm = j-1
    if j==N-1:
        jp = 0
    else:
        jp = j+1
    A[j,[jp,jm]] = -fact # off diagonals
    uexact[j] = math.exp(math.cos(x[j]))
    rhs[j] = uexact[j]*(math.cos(x[j])+math.cos(x[j])**2)
u = spsolve(A, rhs)
error = np.amax(np.absolute(u-uexact))
print 'Maximum norm error %10.4e' % error

## 2D problem: -Lap u + u = f(x,y) with u(x,y) 2 pi periodic in x and y

Second order finite difference method

### Generate the Sparse Matrix, RHS and exact solution

In [ ]:
N = 64
N2 = N**2
h = 2*pi/N
fact = 1/(h**2)
A = csr_matrix((N2,N2))
uexact = np.zeros((N,N))
rhs = np.zeros(N2)
x = np.arange(N)*h


# Inefficient but I don;t know how to input these as diagonals in scipy
for j in range(0,N):
    for i in range(0,N):
        ind = i+N*j # row number
        A[ind,ind] = 4*fact +1 # diagonal term 
        
        if i == N-1:
            indxp = ind-N+1
        else: 
            indxp = ind+1
        
        if i == 0:
            indxm = ind +N-1
        else:
            indxm = ind-1
        
        if j == N-1:
            indyp = ind -N2 +N
        else: 
            indyp = ind+N
        
        if j == 0:
            indym = ind +N2-N
        else:
            indym = ind-N
            
        A[ind,[indxm,indxp,indym,indyp]] = -fact # off diagonals
        xx = x[i]
        yy = x[j]
        cxx = math.cos(xx)
        cyy = math.cos(yy)
        ue = math.exp(cxx+cyy)
        uexact[i,j] = ue
        rhs[ind] = ue*(cxx + cxx**2 + cyy + cyy**2-1)
print 'done'

### Sparse Solve and error calculation

In [ ]:
uvec = spsolve(A, rhs)
ugrid = uvec.reshape(N,N).T # turn solution vector to a grid solution for plotting
error = np.amax(np.absolute(ugrid-uexact))
print 'Maximum norm error %10.4e' % error

### Contour plot of the result

In [ ]:
# Contour plots
xv, yv = np.meshgrid(x,x)

plt.figure(2)
CS = plt.contour(xv, yv, ugrid)
plt.clabel(CS, inline=1, fontsize=10)
plt.title('Computed Solution')
plt.show()

plt.figure(3)
CS = plt.contour(xv, yv, uexact)
plt.clabel(CS, inline=1, fontsize=10)
plt.title('Exact Solution')
plt.show()



### Conjugate Gradient Solve

In [ ]:
uvec,cgflag = sc.sparse.linalg.cg(A, rhs, tol=1e-4)
print 'CG routine output flag %d' % cgflag
# uevec = uexact.reshape(N2) # turn exact solution on the grid into a vector

ugrid = uvec.reshape(N,N).T # turn solution vector to a grid solution for plotting
error = np.amax(np.absolute(ugrid-uexact))
print 'Maximum norm error %10.4e' % error
